In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import sympy as spy
import sympy.abc as abc
import joblib

from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error
import sklearn.preprocessing
from tqdm import tqdm

from src.mlhgp import MLHGP
from src.imlhgp import IMLHGP
from src.nnpehgp import NNPEHGP
from src.kshgp import KSHGP
from src.ksmlhgp import KSMLHGP
from src.ksimlhgp import KSIMLHGP
from src.rnmlhgp import RNHGP as RNHGP2
from src.rnimlhgp import RNHGP as RNHGP3
from src import evaluator

## SUPPRESS ALL WARNINGS
import warnings
warnings.filterwarnings("ignore")

# Problem Definition

Temperature of solid sphere immersed in fluid under uncertainty. Described as:

$$
Y = u_2 + u_3 \sum_{i=1}^\infty \frac{4(\sin\eta_i - \eta_i \cos \eta_i)}{2\eta_i - \sin(2\eta_i)} \exp(-\eta_i^2 \pi_2) \frac{\sin(\eta_i u_1)}{\eta_i u_1},
$$

where $\eta_i$ is a solution of:

$$
1 - \eta_i \cot \eta_i = \pi_1, \quad \eta_i \in ((i-1)\pi, i\pi),
$$

where:

$$
\pi_1 = \frac{u_4 u_7}{u_6}, \quad \pi_2 = \frac{u_6 u_5}{u_8 u_9 u_7^2}
$$

>NOTE: We can approximate the infinite sum with the **first four terms** only

Input variable definition:

| Variable | Description                          | Units          | Minimum | Maximum |
|----------|--------------------------------------|----------------|---------|---------|
| $u_1$      | Distance from the center of sphere   | -              | 0       | 1       |
| $u_2$      | Temperature of fluid                 | $K$              | 250     | 270     |
| $u_3$      | Initial sphere temperature           | $K$              | -100    | -30     |
| $u_4$      | Convective heat transfer coefficient | $kgs^{-3}K^{-1}$ | 180     | 210     |

Fixed / random variables definition

| Variable | Description          | Units               | Value/Distribution         |
|----------|----------------------|---------------------|----------------------------|
| $u_5$    | Time                 | $s$                 | 800                        |
| $u_6$    | Thermal conductivity | $kgs^{-3}K^{-1}$    | $\mathcal{N}(65, 10.3^2)$    |
| $u_7$    | Sphere's radius      | $m$                 | $\mathcal{N}(0.1, 0.02^2)$   |
| $u_8$    | Specific heat        | $m^2 s^{-2} K^{-1}$ | $\mathcal{N}(400, 33.3^2)$   |
| $u_9$    | Density              | $kgm^{-3}$          | $\mathcal{N}(8000, 333.3^2)$ |

In [2]:
def pi1(u4, u6, u7):
    return (u4*u7)/u6

def pi2(u5, u6, u7, u8, u9):
    return (u5*u6) / (u8*u9*(u7**2))

In [3]:
def eta_i_fun(pi_1, i):
    lower_limit = (i-1)*np.pi
    upper_limit = i*np.pi
    solnrange = np.linspace(lower_limit, upper_limit, num=5)
    solutions = []

    for x0 in solnrange:
        try:  # Sometimes 0 division error happen when the initial x0 is not right, using try except to bypass the error
            solutions.append(spy.nsolve(1 - pi_1 - (abc.x * spy.cot(abc.x)) , x0))
        except:
            pass

    solutions = np.unique(solutions)
    cond1 = solutions >= lower_limit
    cond2 = solutions <= upper_limit
    solutions = solutions[np.where(cond1 & cond2)]
    if len(solutions) != 1:
        print(f"eta_i solution is not unique, solution: {solutions}, proceed with taking the first solution")
    
    return solutions[0]

In [4]:
def solve_function(u):
    assert len(u) == 9, "Input length must be 9 (There are 9 input variables)"

    pi_1 = pi1(u[3], u[5], u[6])
    pi_2 = pi2(u[4], u[5], u[6], u[7], u[8])

    # Compute summation, approximating by first four terms:
    temp_res = []
    for i in range(1,5):  # i = [1,2,3,4]
        # Compute eta_i
        eta_i = float(eta_i_fun(pi_1, i))  # convert sympy solution to float, otherwise it's not compatible with numpy
        first_term = (4 * (np.sin(eta_i) - (eta_i * np.cos(eta_i)))) / ((2*eta_i) - np.sin(2*eta_i))
        second_term = np.exp(-1 * eta_i**2 * pi_2) * ((np.sin(eta_i*u[0])) / (eta_i*u[0]))
        temp_res.append(first_term * second_term)
    summation = np.sum(temp_res)

    y = u[1] + u[2]*summation

    return y
        

## Training samples

In [5]:
np.random.seed(42)
n_train = 500
x_train = np.random.uniform(low=[0,250,-100,180], high=[1,270,-30,210], size=(n_train,4))
u5 = np.tile(800, (int(n_train),1))  # time variable
u6 = np.random.normal(loc = 65, scale=10.3, size=(n_train,1))  # thermal conductivity
u7 = np.random.normal(loc = 0.1, scale=0.02, size=(n_train,1))  # Sphere's radius
u8 = np.random.normal(loc = 400, scale=33.3, size=(n_train,1))  # Specific heat
u9 = np.random.normal(loc = 8000, scale=333.3, size=(n_train,1))  # Sphere's density
rand_input = np.hstack((u5,u6,u7,u8,u9))
eval_input = np.hstack((x_train, rand_input))

# Evaluate training data
y_train = []
for inp in tqdm(eval_input):
    y_train.append(solve_function(inp))


100%|██████████| 500/500 [00:06<00:00, 80.73it/s]


In [6]:
y_train

[256.28038102851417,
 230.54789028870138,
 251.18802192763482,
 236.46368788494954,
 250.11031738097165,
 225.71400475483432,
 242.46762874131647,
 232.75763917740778,
 259.31335637321945,
 238.14785954931477,
 232.21434899361668,
 243.1739110105378,
 246.7626536984353,
 257.69980506581805,
 247.40255612397553,
 243.58764115981037,
 222.96527431698382,
 252.5336020124075,
 250.45669540795663,
 225.95659143791823,
 241.78304388677884,
 241.0591387771094,
 237.12426532002203,
 251.04880265492835,
 218.9567117846783,
 231.414210231511,
 238.32519972368067,
 227.27716319813086,
 252.23267044675435,
 242.46390830070183,
 257.6484895863195,
 255.31018864278138,
 241.25083164419445,
 251.29039726279154,
 251.93093908913167,
 238.67384436624465,
 240.122836256936,
 240.9066982172122,
 249.49678414795673,
 241.0996621500819,
 245.34631786991937,
 251.6904453986913,
 238.98085758615994,
 242.1654216025556,
 246.76707324879595,
 246.30038858635973,
 256.2916010750173,
 226.38239019363675,
 246.58

## Testing samples

In [5]:
n_test = 1000  # Number of testing points
n_realization = 10000  # Number of realization for each testing point, to obtain distribution

## make x_test input for the first time
# x_test = np.random.uniform(low=[0,250,-100,180], high=[1,270,-30,210], size=(n_test,4))
# joblib.dump(x_test, "data/sphere_heat_test.pkl")

## make random input for the first time
# u5 = np.tile(800, (int(n_realization),1))  # time variable
# u6 = np.random.normal(loc = 65, scale=10.3, size=(n_realization,1))  # thermal conductivity
# u7 = np.random.normal(loc = 0.1, scale=0.02, size=(n_realization,1))  # Sphere's radius
# u8 = np.random.normal(loc = 400, scale=33.3, size=(n_realization,1))  # Specific heat
# u9 = np.random.normal(loc = 8000, scale=333.3, size=(n_realization,1))  # Sphere's density
# rand_input = np.hstack((u5,u6,u7,u8,u9))
# joblib.dump(rand_input, "data/sphere_randinput_test.pkl")

x_test = joblib.load("data/sphere_heat_test.pkl") #just load the x_test input data
rand_input = joblib.load("data/sphere_randinput_test.pkl") #just load the x_test input data

In [6]:
y_mean = []
y_std = []
for i in tqdm(range(900,1000)):
    xi = x_test[i,:]
    input_component = np.tile(xi, (n_realization,1))
    u_realization = np.hstack((input_component,rand_input))
    y_dist = []
    for u_r in u_realization:
        y = solve_function(u_r)
        y_dist.append(y)
    y_mean.append(np.mean(y_dist))
    y_std.append(np.std(y_dist))
    yres = np.hstack((np.array(y_mean).reshape(-1,1), np.array(y_std).reshape(-1,1)))

    if (i+1)%10 == 0:
        print(f"saving at loop {i+1}")
        joblib.dump(yres,"output/sphere_result_901_1000.pkl")


 10%|█         | 10/100 [19:28<2:54:41, 116.46s/it]

saving at loop 910


 20%|██        | 20/100 [38:56<2:37:04, 117.80s/it]

saving at loop 920


 30%|███       | 30/100 [58:46<2:16:26, 116.95s/it]

...

saving at loop 980


 90%|█████████ | 90/100 [2:55:15<19:22, 116.26s/it]

saving at loop 990


100%|██████████| 100/100 [3:15:03<00:00, 117.04s/it]

saving at loop 1000


In [8]:
x_test.shape

(1000, 4)

In [ ]:
name = ["1_100","101_300","301_500","501_700","701_900","901_1000"]